In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer, execute
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.extensions.standard import HGate, SGate, SdgGate, XGate
import numpy as np
from qiskit.quantum_info.states.measures import state_fidelity
import copy
import itertools
import cutter
import uniter_sv as uniter
import MIQCP_searcher as searcher
from qcg.generators import gen_supremacy, gen_hwea

def reverseBits(num,bitSize):
    binary = bin(num)
    reverse = binary[-1:1:-1]
    reverse = reverse + (bitSize - len(reverse))*'0'
    return int(reverse,2)

def simulate_circ(circ, simulator='statevector_simulator'):
    backend = BasicAer.get_backend(simulator)
    job = execute(circ, backend=backend)
    result = job.result()
    outputstate = result.get_statevector(circ)
    outputstate_ordered = [0 for sv in outputstate]
    for i, sv in enumerate(outputstate):
        reverse_i = reverseBits(i,len(circ.qubits))
        outputstate_ordered[reverse_i] = sv
    return outputstate_ordered

q = QuantumRegister(4,'q')
circ = QuantumCircuit(q)
circ.h(q)
circ.cz(0,1)
circ.rx(np.pi/2,q[:2])
circ.cz(0,2)
circ.rx(np.pi/2,0)
circ.h(0)
circ.cz(2,3)
circ.rx(np.pi/2,q[2:4])
circ.cz(1,3)

uncut_circ_sv = simulate_circ(circ)
print('original circuit:')
print(circ)
print('uncut circuit state vector:', uncut_circ_sv)
print('*'*100)

positions = [(circ.qubits[0], 1), (circ.qubits[3], 1)]
print('cut at:',positions)
clusters, complete_path_map, K, d = cutter.cut_circuit(circ, positions)

print('Complete path map:')
[print(x, ': ',complete_path_map[x]) for x in complete_path_map]
for idx, x in enumerate(clusters):
    print('cluster %d circuit:'%idx)
    print(x)
print('*'*50)

original circuit:
        ┌───┐   ┌──────────┐   ┌──────────┐   ┌───┐       
q_0: |0>┤ H ├─■─┤ Rx(pi/2) ├─■─┤ Rx(pi/2) ├───┤ H ├───────
        ├───┤ │ ├──────────┤ │ └──────────┘   └───┘       
q_1: |0>┤ H ├─■─┤ Rx(pi/2) ├─┼──────────────────────────■─
        ├───┤   └──────────┘ │             ┌──────────┐ │ 
q_2: |0>┤ H ├────────────────■──────■──────┤ Rx(pi/2) ├─┼─
        ├───┤                       │      ├──────────┤ │ 
q_3: |0>┤ H ├───────────────────────■──────┤ Rx(pi/2) ├─■─
        └───┘                              └──────────┘   
uncut circuit state vector: [(0.35355339059327373+0j), 0j, 0j, -0.35355339059327373j, 0j, 0.35355339059327373j, (-0.35355339059327373+0j), 0j, 0.35355339059327373j, 0j, 0j, (0.35355339059327373+0j), 0j, (0.35355339059327373+0j), 0.35355339059327373j, 0j]
****************************************************************************************************
cut at: [(Qubit(QuantumRegister(4, 'q'), 0), 1), (Qubit(QuantumRegister(4, 'q'), 3), 1)]
Comple

In [4]:
def project_sv(cluster_sv,projection):
    projected = []
    for i, sv in enumerate(cluster_sv):
        bin_i = bin(i)[2:].zfill(len(projection))
        pattern_match = True
        for b, p in zip(bin_i, projection):
            b = int(b)
            if b!=p and p!='x':
                pattern_match = False
                break
        if pattern_match:
            print('state retained:',bin_i)
            projected.append(sv)
    return projected

def init_sv(circ,inits):
    circ_copy = copy.deepcopy(circ)
    circ_dag = circuit_to_dag(circ_copy)
    for q_i, q in enumerate(circ.qubits):
        if inits[q_i] == 1:
            circ_dag.apply_operation_front(op=XGate(),qargs=[q],cargs=[])
    circ = dag_to_circuit(circ_dag)
    print(circ)
    sv = simulate_circ(circ)
    return sv  
            
# (0,0) case
cluster_0_sv = init_sv(clusters[0],[0,0,0])
cluster_0_projected = project_sv(cluster_0_sv,[0,'x','x'])

cluster_1_sv = init_sv(clusters[1],[0,0,0])
cluster_1_projected = project_sv(cluster_1_sv,['x','x',0])

summation_term_0 = np.kron(cluster_0_projected,cluster_1_projected)
print('-'*100)

# (0,1) case
cluster_0_sv = init_sv(clusters[0],[0,0,1])
cluster_0_projected = project_sv(cluster_0_sv,[0,'x','x'])

cluster_1_sv = init_sv(clusters[1],[0,0,0])
cluster_1_projected = project_sv(cluster_1_sv,['x','x',1])

summation_term_1 = np.kron(cluster_0_projected,cluster_1_projected)
print('-'*100)

# (1,0) case
cluster_0_sv = init_sv(clusters[0],[0,0,0])
cluster_0_projected = project_sv(cluster_0_sv,[1,'x','x'])

cluster_1_sv = init_sv(clusters[1],[1,0,0])
cluster_1_projected = project_sv(cluster_1_sv,['x','x',0])

summation_term_2 = np.kron(cluster_0_projected,cluster_1_projected)
print('-'*100)

# (1,1) case
cluster_0_sv = init_sv(clusters[0],[0,0,1])
cluster_0_projected = project_sv(cluster_0_sv,[1,'x','x'])

cluster_1_sv = init_sv(clusters[1],[1,0,0])
cluster_1_projected = project_sv(cluster_1_sv,['x','x',1])

summation_term_3 = np.kron(cluster_0_projected,cluster_1_projected)

reconstructed = summation_term_0 + summation_term_1 + summation_term_2 + summation_term_3
print('-'*100)

print(reconstructed)
print(uncut_circ_sv)
print('*'*100)

reconstructed = uniter.reconstructed_reorder(reconstructed,complete_path_map)
print('reconstruction fidelity =',state_fidelity(reconstructed,original_circ_sv))

           ┌───┐                      
q_0: |0>───┤ H ├─────■────────────────
           ├───┤     │ ┌──────────┐   
q_1: |0>───┤ H ├─────■─┤ Rx(pi/2) ├─■─
        ┌──┴───┴───┐   └──────────┘ │ 
q_2: |0>┤ Rx(pi/2) ├────────────────■─
        └──────────┘                  
state retained: 000
state retained: 001
state retained: 010
state retained: 011
        ┌──────────┐   ┌──────────┐   ┌───┐    
q_0: |0>┤ Rx(pi/2) ├─■─┤ Rx(pi/2) ├───┤ H ├────
        └──┬───┬───┘ │ └──────────┘┌──┴───┴───┐
q_1: |0>───┤ H ├─────■──────■──────┤ Rx(pi/2) ├
           ├───┤            │      └──────────┘
q_2: |0>───┤ H ├────────────■──────────────────
           └───┘                               
state retained: 000
state retained: 010
state retained: 100
state retained: 110
----------------------------------------------------------------------------------------------------
        ┌───┐                           
q_0: |0>┤ H ├─────■─────────────────────
        ├───┤     │      ┌──────────┐   
q_1: |0